In [1]:
import os
import pathlib
import shutil
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
import tensorflow_datasets as tfds
from keras import regularizers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print(tf.__version__)

2.8.0


In [2]:
def create_label_dir(df):
    class_names = np.sort(df['label'].unique())
    number_of_classes = len(class_names)

    if not os.path.exists('../gdsc-ai-challenge/train'):
        return class_names, number_of_classes

    for class_name in class_names:
        dir = pathlib.Path('../gdsc-ai-challenge/train/' + class_name)
        if dir.exists():
            continue
        else:
            dir.mkdir()
    
    return class_names, number_of_classes

def sort_data(df):
    if not os.path.exists('../gdsc-ai-challenge/train'):
        return
        
    for image_dir in [str(img) for img in list(pathlib.Path('../gdsc-ai-challenge/train').glob('*.png'))]:
        id = int(image_dir.removeprefix('../gdsc-ai-challenge/train/').removesuffix('.png'))
        label = df['label'][id - 1]
        dest_path = os.path.join('../gdsc-ai-challenge/train', label, str(id) + '.png')
        shutil.move(image_dir, dest_path)

In [3]:
label_df = pd.read_csv('../gdsc-ai-challenge/trainLabels.csv')

class_names, number_of_classes = create_label_dir(label_df)
sort_data(label_df)

In [4]:
datagen = ImageDataGenerator(
    validation_split=0.3,
    rotation_range=15,
    zoom_range=0.3,
    horizontal_flip=True,
    rescale=1.0/255,
    fill_mode='nearest'
)

train_gen = datagen.flow_from_directory(
    '../gdsc-ai-challenge/train',
    target_size=(32,32),
    batch_size=10,
    subset='training',
    seed=42,
    color_mode='grayscale',
    class_mode='sparse'
)

val_gen = datagen.flow_from_directory(
    '../gdsc-ai-challenge/train',
    target_size=(32,32),
    batch_size=10,
    subset='validation',
    seed=42,
    color_mode='grayscale',
    class_mode='sparse'
)

print(val_gen[0][1][0])

print(val_gen.classes.shape)

Found 35004 images belonging to 10 classes.
Found 14996 images belonging to 10 classes.
8.0
(14996,)


In [5]:
model = keras.models.Sequential([
    keras.layers.Input((32,32,1)),
    keras.layers.Conv2D(8, (5,5), padding='same',
                        kernel_regularizer=regularizers.l2(1e-4),
                        activity_regularizer=regularizers.l2(1e-4),
                        kernel_initializer='he_normal',
                        activation='elu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(8, (5,5), padding='same',
                        kernel_regularizer=regularizers.l2(1e-4),
                        activity_regularizer=regularizers.l2(1e-4),
                        kernel_initializer='he_normal',
                        activation='elu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Conv2D(16, (3,3), padding='same',
                        kernel_regularizer=regularizers.l2(1e-4),
                        activity_regularizer=regularizers.l2(1e-4),
                        kernel_initializer='he_normal',
                        activation='elu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(16, (3,3), padding='same',
                        kernel_regularizer=regularizers.l2(1e-4),
                        activity_regularizer=regularizers.l2(1e-4),
                        kernel_initializer='he_normal',
                        activation='elu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Conv2D(32, (3,3), padding='same',
                        kernel_regularizer=regularizers.l2(1e-4),
                        activity_regularizer=regularizers.l2(1e-4),
                        kernel_initializer='he_normal',
                        activation='elu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(32, (3,3), padding='same',
                        kernel_regularizer=regularizers.l2(1e-4),
                        activity_regularizer=regularizers.l2(1e-4),
                        kernel_initializer='he_normal',
                        activation='elu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(32, (3,3), padding='same',
                        kernel_regularizer=regularizers.l2(1e-4),
                        activity_regularizer=regularizers.l2(1e-4),
                        kernel_initializer='he_normal',
                        activation='elu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Dropout(0.2),
    keras.layers.Flatten(),
    keras.layers.Dense(number_of_classes, activation='softmax')
])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
                                              loss='sparse_categorical_crossentropy',
                                              metrics=['accuracy'])

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-03-13 03:25:57.204553: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-13 03:25:57.204663: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 8)         208       
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 8)        32        
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 8)         1608      
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 8)        32        
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 8)        0         
 )                                                               
                                                        

In [7]:
class stopCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}) :
        if (logs.get('val_accuracy') >= 0.999 and logs.get('accuracy') >= 0.999) :
            print('\nReached 99.9% accuracy so stopping training')
            self.model.stop_training = True

In [8]:
callback = stopCallback()

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath='../Model/aiseries/version1.hdf5',
                                                               monitor='val_loss',
                                                               mode='min',
                                                               save_best_only=True)

history = model.fit(train_gen, 
                    epochs=1000, 
                    callbacks=[callback, model_checkpoint_callback], 
                    validation_data=val_gen)

Epoch 1/1000


2022-03-13 03:25:57.619035: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-13 03:25:58.172077: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


3499/3501 [============================>.] - ETA: 0s - loss: 3.8927 - accuracy: 0.2412

2022-03-13 03:27:26.791189: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


3501/3501 [==============================] - 103s 28ms/step - loss: 3.8925 - accuracy: 0.2412 - val_loss: 3.1648 - val_accuracy: 0.3130
Epoch 2/1000
3501/3501 [==============================] - 99s 28ms/step - loss: 3.0236 - accuracy: 0.3176 - val_loss: 2.8036 - val_accuracy: 0.3517
Epoch 3/1000
3501/3501 [==============================] - 100s 28ms/step - loss: 2.7304 - accuracy: 0.3568 - val_loss: 2.5936 - val_accuracy: 0.3810
Epoch 4/1000
3501/3501 [==============================] - 99s 28ms/step - loss: 2.5445 - accuracy: 0.3880 - val_loss: 2.4675 - val_accuracy: 0.3923
Epoch 5/1000
3501/3501 [==============================] - 102s 29ms/step - loss: 2.3943 - accuracy: 0.4084 - val_loss: 2.2868 - val_accuracy: 0.4360
Epoch 6/1000
3501/3501 [==============================] - 100s 28ms/step - loss: 2.2790 - accuracy: 0.4312 - val_loss: 2.2759 - val_accuracy: 0.4320
Epoch 7/1000
3501/3501 [==============================] - 99s 28ms/step - loss: 2.1862 - accuracy: 0.4463 - val_loss: 2.2

In [1]:
plt.plot(history.history['loss'], color='orange', label='Train loss')
plt.plot(history.history['val_loss'], color='blue', label='Validation loss')
plt.legend()
plt.show()

NameError: name 'plt' is not defined